# Generator Danych Testowych

## Import modułów i bibliotek

Skrypt wykorzystuje następujące moduły i biblioteki
- [Faker](https://faker.readthedocs.io/en/master/),
- [random](https://docs.python.org/3/library/random.html),
- [pandas](https://pandas.pydata.org/docs/).

In [1]:
from random import random, uniform

import pandas as pd
from faker import Faker

fake = Faker("pl_PL")

## Podstawowe ustawienia

W słowniku `city` można ustawić nazwę (`name`) gminy, dla której chcemy wygenerować dane oraz zasięg (`y_min_max` oraz `x_min_max`), w którym mają zostać wygenerowane współrzędne punktowe. Zasięg można obliczyć w QGIS na podstawie granic gminy. Dane dla każdego z wymiarów podawane się w krotce (tuple), w której pierwsza liczba to minimum a druga maksimum.

Krotka `sample_categories` zawiera nazwy kategorii, które będą wykorzystane przy generowaniu danych.

Zmienna `desired_samples` określa liczbę próbek, które mają zostać wygenerowane. **NB.** Należy wpisać 50%-60% więcej niż ma się faktycznie znaleźć w granicach. Po wygenrowaniu pliku należy go przyciąć do granic w QGIS.

Zmienna `dev_mode` to flaga. Jeżeli ustawiona jako prawda (`True`), plik .csv nie zostanie zapisany, w przeciwnym wypadku (`False`) plik zostanie zapisany.

In [2]:
city: dict[str, str | tuple[float | int, float | int]] = {
    "name": "Siemianowice Śląskie",
    "y_min_max": (50.274825, 50.349919),
    "x_min_max": (18.98523, 19.061271)
}

sample_categories: tuple[str, ...] = ("administracja",
                                      "dostępność",
                                      "infrastruktura",
                                      "kultura, sport i rekreacja",
                                      "ochrona środowiska",
                                      "oświata",
                                      "transport")

desired_samples: int = 200

dev_mode: bool = False

## Generowanie danych

### Generowanie współrzędnych x i y

In [3]:
def generate_coords(y_or_x: str,
                    num_to_generate: int = desired_samples,
                    extent_y: tuple[float | int, float | int] = city["y_min_max"],
                    extent_x: tuple[float | int, float | int] = city["x_min_max"]) -> list[float | None]:
    match y_or_x:
        case "y":
            y: list[float] = []
            min_y, max_y = extent_y
            for _ in range(num_to_generate):
                y.append(uniform(min_y, max_y))
            return y
        case "x":
            x: list[float] = []
            min_x, max_x = extent_x
            for _ in range(num_to_generate):
                x.append(uniform(min_x, max_x))
            return x
        # In any other case return an empty list:
        case _:
            return [None] * num_to_generate

### Generowanie pozostałych danych

In [4]:
def generate_test_data(data_type: str,
                       num_to_generate: int = desired_samples,
                       max_num_chars: int = 256,
                       no_dot_at_end: bool = True,
                       single_city: bool = True) -> list:
    match data_type:
        case "texts":
            texts: list[str] = fake.texts(nb_texts=num_to_generate, max_nb_chars=max_num_chars)
            if no_dot_at_end:
                for i in range(len(texts)):
                    texts[i] = texts[i][:-1]
            return texts
        case "categories":
            return fake.random_choices(elements=sample_categories, length=num_to_generate)
        case "prefixes":
            return fake.random_choices(elements=("ul.", "al.", "pl."), length=num_to_generate)
        case "streets":
            s: list[str] = []
            for _ in range(num_to_generate):
                s.append(fake.street_name())
            return s
        case "buildings":
            b: list[str | None] = []
            for _ in range(num_to_generate):
                if random() > 0.33:
                    b.append(fake.building_number())
                else:
                    b.append(None)
            return b
        case "postcodes":
            p: list[str] = []
            for _ in range(num_to_generate):
                p.append(fake.postcode())
            return p
        case "cities":
            c: list[str] = []
            if single_city:
                c.append(city["name"])
                return c * num_to_generate
            else:
                for _ in range(num_to_generate):
                    c.append(fake.city())
                return c
        case "dates":
            d: list = []
            for _ in range(num_to_generate):
                d.append(fake.date_between(start_date="-10y", end_date="-6m"))
            return d
        case "years":
            y: list[int] = []
            for _ in range(num_to_generate):
                y.append(fake.year())
            return y
        case "amounts":
            a: list[float] = []
            for _ in range(num_to_generate):
                a.append(fake.pyfloat(left_digits=7, right_digits=2, positive=True))
            return a
        case "bools":
            b: list[bool] = []
            for _ in range(num_to_generate):
                b.append(fake.pybool())
            return b
        case "urls":
            u: list[str | None] = []
            domain_name: str = fake.safe_domain_name()
            for _ in range(num_to_generate):
                if random() > 0.25:
                    u.append(f"https://{domain_name}/{fake.slug()}")
                else:
                    u.append(None)
            return u
        case _:
            return [None] * num_to_generate

### Struktura tabeli

Słownik `d` zawiera strukturę generowanej tabeli. Każdy element to kolumna tabeli.

In [5]:
d = {
    "foto_sciezka": [None] * desired_samples,  # To be filled in in QGIS with image path
    "kategoria": generate_test_data("categories"),
    "nazwa": generate_test_data("texts", max_num_chars=64),
    "zakres": generate_test_data("texts", max_num_chars=256, no_dot_at_end=False),
    "cecha": generate_test_data("prefixes"),
    "ulica_nazwa": generate_test_data("streets"),
    "nr_budynku": generate_test_data("buildings"),  # Will be empty for approx 1/3 samples, this is expected behaviour
    "kod_pocztowy": generate_test_data("postcodes"),
    "miejscowosc": generate_test_data("cities", single_city=True),  # If multiple city names needed, change `single_city` to False
    "adres": [None] * desired_samples,  # To be filled in at the data cleaning stage
    "data_rozpoczecia": generate_test_data("dates"),
    "data_zakonczenia": generate_test_data("dates"),  # To be cleaned & adjusted at the data cleaning stage
    "czy_zakonczony": generate_test_data("bools"),
    "wartosc": generate_test_data("amounts"),
    "czy_dofinansowany": generate_test_data("bools"),
    "zrodlo_dofinansowania": generate_test_data("texts", max_num_chars=64),  # To be cleaned at the data cleaning stage
    "wartosc_dofinansowania": generate_test_data("amounts"),  # To be cleaned & adjusted at the data cleaning stage
    "czy_bo": generate_test_data("bools"),
    "edycja_bo": generate_test_data("years"),  # To be cleaned at the data cleaning stage
    "url_fiszka": generate_test_data("urls"),  # Will be empty for approx 1/4 samples, this is expected behaviour
    "url_geoportal": generate_test_data("urls"),  # Will be empty for approx 1/4 samples, this is expected behaviour
    "url_mapa": generate_test_data("urls"),  # Will be empty for approx 1/4 samples, this is expected behaviour
    "popup_url_html": [None] * desired_samples,  # To be filled in at the data cleaning stage
    "y": generate_coords("y"),
    "x": generate_coords("x")
}

## Oczyszczanie danych testowych

Wygenerowane dane wymagają oczyszczenie i uzupełnień przed eksportem.

Funkcja `fill_address_column()` populuje kolumnę `adres` na podstawie wygenerowanych danych cząstkowych.

Funkcja `adjust_aid_received()` dostosowuje wartość dofinansowania.

Funkcja `adjust_end_dates()` dostosowuje daty zakończenia.

Funkcja `fill_popup_url_html()` generuje kod html dla linków, który będzie wyświetlony w popupie.

Funkcja `clean_unwanted_data()` usuwa wartości, które powinny być puste (np. datę zakończenia niezakończonych inwestycji).

In [6]:
def fill_address_column() -> None:
    for index in range(len(df.index)):
        if df.at[index, "nr_budynku"] != None:
            full_address: str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]} {df.at[index, "nr_budynku"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        else:
            full_address: str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        df.at[index, "adres"] = full_address


def adjust_aid_received(min_spread: float = 0.65, max_spread: float = 0.85) -> None:
    for index in range(len(df.index)):
        df.at[index, "wartosc_dofinansowania"] = round(df.at[index, "wartosc"] * uniform(min_spread, max_spread), 2)


def adjust_end_dates() -> None:
    for index in range(len(df.index)):
        if df.at[index, "data_zakonczenia"] < df.at[index, "data_rozpoczecia"]:
            df.at[index, "data_zakonczenia"] = fake.date_between_dates(date_start=df.at[index, "data_rozpoczecia"], date_end="-1m")


def fill_popup_url_html() -> None:
    for index in range(len(df.index)):
        output_html: str | None = ""
        links: list[tuple[str | None, str]] = [(df.at[index, "url_fiszka"], "Więcej"),
                                               (df.at[index, "url_geoportal"], "Geoportal"),
                                               (df.at[index, "url_mapa"], "Mapa")]
        if (links[0][0] is None and links[1][0] is None and links[2][0] is None):
            output_html = None
        else:
            for link in links:
                url, title = link
                if url is not None:
                    output_html += f'<a href="{url}">{title}</a>'
                    output_html = output_html.replace("</a><a", "</a> | <a")
                    if len(output_html) > 256:
                        output_html = None
        df.at[index, "popup_url_html"] = output_html


def clean_unwanted_data() -> None:
    df.loc[df["czy_zakonczony"] == False, "data_zakonczenia"] = None
    df.loc[df["czy_dofinansowany"] == False, "zrodlo_dofinansowania"] = None
    df.loc[df["czy_dofinansowany"] == False, "wartosc_dofinansowania"] = None
    df.loc[df["czy_bo"] == False, "edycja_bo"] = None

In [7]:
df = pd.DataFrame(data=d)
fill_address_column()
adjust_aid_received()
adjust_end_dates()
fill_popup_url_html()
clean_unwanted_data()
df

,foto_sciezka,kategoria,nazwa,zakres,cecha,ulica_nazwa,nr_budynku,kod_pocztowy,miejscowosc,adres,...,zrodlo_dofinansowania,wartosc_dofinansowania,czy_bo,edycja_bo,url_fiszka,url_geoportal,url_mapa,popup_url_html,y,x
0,None,administracja,Biec prawdziwy nosić. Często jeżeli pismo ostatni,Pierwiastek środa czekolada papier. Wolny koro...,al.,Rózana,None,06-518,Siemianowice Śląskie,al. Rózana; 06-518 Siemianowice Śląskie,...,Jednostka ozdoba włożyć więc dolny rosnąć klub,824995.53,True,1977,https://example.org/blady,https://example.net/tanczyc-don,https://example.org/od-blisko,"<a href=""https://example.org/blady"">Więcej</a>...",50.342542,19.020487
1,None,dostępność,Wycieczka jednocześnie zwrot spadać królowa,Wyrób umieszczać norma instrument postawa pomi...,ul.,Topolowa,85,14-957,Siemianowice Śląskie,ul. Topolowa 85; 14-957 Siemianowice Śląskie,...,Fizyczny wnętrze ochrona rosyjski. Wieś przech...,3064522.75,False,None,https://example.org/dodac-inny-zamiar,https://example.net/szef,https://example.org/w-oznaczac-kobiecy,"<a href=""https://example.org/dodac-inny-zamiar...",50.287530,19.008678
2,None,ochrona środowiska,Strona przecież inaczej ani nie- powietrze,Wóz spadek zdanie Australia. Rysunek piłka rel...,pl.,Poziomkowa,78/32,17-274,Siemianowice Śląskie,pl. Poziomkowa 78/32; 17-274 Siemianowice Śląskie,...,None,NaN,False,None,https://example.org/jezus-republika,None,https://example.org/japonia-slad,"<a href=""https://example.org/jezus-republika"">...",50.319892,19.006276
3,None,ochrona środowiska,Niezwykły Egipt dopływ mama postać dziewczynka,Budowa biec dość cztery komputer Kościół. Wiat...,al.,Orzechowa,None,27-813,Siemianowice Śląskie,al. Orzechowa; 27-813 Siemianowice Śląskie,...,None,NaN,False,None,https://example.org/warstwa-dziura,https://example.net/obcy-potrzebny,https://example.org/drogi-wydawac-sie,"<a href=""https://example.org/warstwa-dziura"">W...",50.296314,19.039647
4,None,infrastruktura,Styczeń pewny wolność smutek białoruski opuści...,Ptak odkryć wyścig środek księga obywatel. Urz...,ul.,Jaworowa,None,55-035,Siemianowice Śląskie,ul. Jaworowa; 55-035 Siemianowice Śląskie,...,None,NaN,False,None,None,None,https://example.org/fabryka-krolewski,"<a href=""https://example.org/fabryka-krolewski...",50.285767,19.059823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,None,transport,Zobaczyć kolorowy sól waluta,Zaraz mnóstwo głos chleb żyć martwy piec.\nDźw...,ul.,Wiklinowa,85,49-693,Siemianowice Śląskie,ul. Wiklinowa 85; 49-693 Siemianowice Śląskie,...,Telefon wioska powstawać członek główny dach,5485646.60,False,None,https://example.org/czasem-ogien,None,https://example.org/wiosna-interes-miy,"<a href=""https://example.org/czasem-ogien"">Wię...",50.280514,19.002086
196,None,administracja,Mama chrześcijański okno wchodzić. Urodzić Się...,Tutaj wielkość zaś niemiecki dostać szyja. Poz...,pl.,Turkusowa,68/63,82-613,Siemianowice Śląskie,pl. Turkusowa 68/63; 82-613 Siemianowice Śląskie,...,Ser handlowy panować rzeczywistość dzieło piór...,3130716.46,True,1998,https://example.org/ogien-marka-szczyt,None,None,"<a href=""https://example.org/ogien-marka-szczy...",50.279818,19.043472
197,None,dostępność,Kość owad alfabet wojsko matka wiersz,Talerz przejście owoc podać. 80 myśleć zachowa...,pl.,Wyszyńskiego,721,56-972,Siemianowice Śląskie,pl. Wyszyńskiego 721; 56-972 Siemianowice Śląskie,...,Morze Śródziemne moc nadzieja kto noc. Belgia ...,3595165.65,True,2008,https://example.org/znajdowac-byc,https://example.net/zywy-walczyc,https://example.org/lista-ona,"<a href=""https://example.org/znajdowac-byc"">Wi...",50.302131,19.018567
198,None,administracja,Zapalenie parlament muzyka przepis,Otaczać ulegać robić przypadek 5. Budynek góra...,al.,Krakowska,84,24-898,Siemianowice Śląskie,al. Krakowska 84; 24-898 Siemianowice Śląskie,...,Policzek sport rzucać odmiana,2149630.17,True,2015,https://example.org/potem-zodziej,None,https://example.org/pora-zimny-morski,"<a href=""https://example.org

## Eksport danych testowych do pliku

Jeżeli dane się nie zapisują, sprawdź wartość flagi `dev_mode` w sekcji *Podstawowe ustawienia* - powinna zostać ustawiona, jako `False`.

In [8]:
if not dev_mode:
    df.to_csv('test_data_set.csv', index=False)